# Risk Scoring Model — Invisible City

**Goal**: Assign each facility a risk score based on how much they release, what they release, how often, and where.

- Feature engineering: total emissions, frequency, toxicity proxy
- Normalize features, combine into single score
- Optional: fit regression to learn weights
- Plot risk score distribution

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

BASE = os.getcwd()
PROC = os.path.join(BASE, 'data', 'processed')
OUT = os.path.join(BASE, 'output')
os.makedirs(OUT, exist_ok=True)
os.makedirs(os.path.join(OUT, 'figures'), exist_ok=True)

## 1. Load cleaned data and releases (for toxicity)

In [ ]:
facilities = pd.read_csv(os.path.join(PROC, 'facilities_clean.csv'))
releases = pd.read_csv(os.path.join(PROC, 'releases_clean.csv'))

# Simple toxicity proxy (1-10) per chemical (match cleaned chemical_name)
TOXICITY = {
    'Ammonia': 4, 'Benzene': 9, 'Lead': 10, 'Sulphuric Acid': 6, 'Toluene': 5,
    'Xylene': 5, 'Zinc': 4, 'Particulate Matter': 6, 'Vocs': 5, 'Nitrogen Oxides': 5
}
releases['toxicity'] = releases['chemical_name'].map(TOXICITY).fillna(5)
tox_by_facility = releases.groupby('facility_id').agg(
    mean_toxicity=('toxicity', 'mean'),
    max_toxicity=('toxicity', 'max')
).reset_index()
facilities = facilities.merge(tox_by_facility, on='facility_id', how='left')
facilities['mean_toxicity'] = facilities['mean_toxicity'].fillna(5)
facilities['max_toxicity'] = facilities['max_toxicity'].fillna(5)

## 2. Feature engineering

In [ ]:
df = facilities.copy()
df['log_release'] = np.log1p(df['total_release_kg'])
df['release_frequency'] = df['release_count']
df['toxicity_proxy'] = df['mean_toxicity'] * 0.5 + df['max_toxicity'] * 0.5

features = ['log_release', 'release_frequency', 'toxicity_proxy']
X = df[features].copy()
X['release_frequency'] = np.log1p(X['release_frequency'])

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=features, index=df.index)
df[features] = X_scaled

## 3. Risk score: weighted sum (interpretable)

Risk ≈ amount × toxicity × frequency. We use normalized features and weights: 0.5 amount, 0.3 toxicity, 0.2 frequency.

In [ ]:
weights = {'log_release': 0.5, 'toxicity_proxy': 0.3, 'release_frequency': 0.2}
df['risk_score'] = sum(df[k] * w for k, w in weights.items())
df['risk_score'] = (df['risk_score'] - df['risk_score'].min()) / (df['risk_score'].max() - df['risk_score'].min() + 1e-9) * 100
df['risk_score'] = df['risk_score'].round(2)

df.to_csv(os.path.join(PROC, 'facilities_with_risk.csv'), index=False)
print('Risk score range:', df['risk_score'].min(), '-', df['risk_score'].max())
display(df[['facility_id', 'industry', 'total_release_kg', 'risk_score']].head(10))

## 4. Distribution of risk scores

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(df['risk_score'], bins=30, color='steelblue', edgecolor='white', alpha=0.8)
ax.axvline(df['risk_score'].quantile(0.95), color='coral', linestyle='--', label='Top 5%')
ax.set_xlabel('Risk score')
ax.set_ylabel('Count')
ax.set_title('Distribution of facility risk scores')
ax.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUT, 'figures', 'risk_distribution.png'), dpi=150)
plt.show()